In [1]:
!nvidia-smi

Mon Dec 12 11:27:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.154                Driver Version: 390.154                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:73:00.0  On |                  N/A |
| 26%   44C    P2    56W / 250W |    814MiB / 11169MiB |     16%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Import libraries 

In [2]:
import os

In [3]:
os.chdir('/home/ali/RELAXIN/alphafold')

In [11]:
ls

insane_altered.py*                 step6.1_minimization.mdp
martini_v3.0.0.itp                 step6.2_equilibration.mdp*
martini_v3.0_lab_nf.itp            step6.3_equilibration.mdp*
minim.mdp*                         step6.4_equilibration.mdp*
modele_oriented_proa_modified.itp  step6.5_equilibration.mdp*
model_start_aligned.pdb            step6.6_equilibration.mdp*
rec_relax.pdb                      step7_production.mdp*
step6.0_minimization.mdp


# Prepare the system

In [13]:
!martinize2 -f model_start_aligned.pdb -ff martini3001 -x start_cg.pdb -o topoltemp.top -dssp /usr/bin/dssp -elastic -scfix -cys 0.40 -p backbone -pf 1000

    INFO - general - Read 1 molecules from PDB file model_start_aligned.pdb
    INFO - step - Guessing the bonds.
    INFO - general - 2 molecules after guessing bonds
    INFO - step - Repairing the graph.
    INFO - general - Applying modification N-ter to residue A-GLY20
    INFO - general - Applying modification C-ter to residue A-GLN710
    INFO - missing-atom - Missing atom GLN710:OXT
    INFO - general - Applying modification N-ter to residue G-LEU1
    INFO - general - Applying modification N-ter to residue X-SER24
    INFO - general - Applying modification C-ter to residue X-SER48
    INFO - missing-atom - Missing atom SER48:OXT
    INFO - step - Dealing with modifications.
    INFO - general - Identified the modifications ['N-ter'] on residues ['GLY20', 'GLY20', 'GLY20', 'GLY20']
    INFO - general - Identified the modifications ['C-ter'] on residues ['GLN710', 'GLN710', 'GLN710']
    INFO - general - Identified the modifications ['N-ter'] on residues ['LEU1', 'LEU1', 'LEU1',

In [34]:
!python2.7 insane_altered.py -salt 0.15 -charge -0 -sol W -f start_cg.pdb -o system.gro -pbc rectangular -d 30 -x 15 -y 15 -z 22 -l POPC:100 -p topol.top

; X: 15.000 (19 bins) Y: 15.000 (19 bins) in upper leaflet
; X: 15.000 (19 bins) Y: 15.000 (19 bins) in lower leaflet
; 342 lipids in upper leaflet, 336 lipids in lower leaflet
; NDX Solute 1 1767
; Charge of protein: 13.000000
; NDX Membrane 1768 9903
; Charge of membrane: 0.000000
; Total charge: 13.000000
; NDX Solvent 9904 42203
; NDX System 1 42203
; "I mean, the good stuff is just INSANE" --Julia Ormond


In [35]:
!sed -i -e "s/Protein        1/PROA 1\nmolecule_1 1/g" ./topol.top

In [36]:
!cat topol.top

#include "martini_v3.0.0.itp"
#include "martini_v3.0_lab_nf.itp"
#include "molecule_0.itp"
#include "molecule_1.itp"

[ system ]
; name
Protein in INSANE! Membrane UpperLeaflet>POPC=100.0 LowerLeaflet>POPC=100.0

[ molecules ]
; name  number
PROA 1
molecule_1 1
POPC           342
POPC           336
W            31605
NA+            341
CL-            354


# Energy Minimization

In [37]:
!gmx_mpi grompp -f step6.0_minimization.mdp -o step6.0_minimization.tpr -c system.gro -r system.gro -p topol.top

                      :-) GROMACS - gmx grompp, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

In [38]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.0_minimization -ntomp 5 -v 

                      :-) GROMACS - gmx mdrun, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
      

Step=   89, Dmax= 1.3e-02 nm, Epot= -3.70811e+05 Fmax= 1.32932e+04, atom= 1480
Step=   90, Dmax= 1.6e-02 nm, Epot= -3.72142e+05 Fmax= 3.99247e+04, atom= 1480
Step=   91, Dmax= 1.9e-02 nm, Epot= -3.75419e+05 Fmax= 2.33698e+04, atom= 1480
Step=   93, Dmax= 1.1e-02 nm, Epot= -3.77505e+05 Fmax= 1.46504e+04, atom= 1480
Step=   94, Dmax= 1.4e-02 nm, Epot= -3.79044e+05 Fmax= 3.06420e+04, atom= 1480
Step=   95, Dmax= 1.7e-02 nm, Epot= -3.81279e+05 Fmax= 2.42150e+04, atom= 1480
Step=   96, Dmax= 2.0e-02 nm, Epot= -3.81717e+05 Fmax= 4.09849e+04, atom= 1480
Step=   97, Dmax= 2.4e-02 nm, Epot= -3.83855e+05 Fmax= 3.80404e+04, atom= 1480
Step=   99, Dmax= 1.4e-02 nm, Epot= -3.87237e+05 Fmax= 9.43338e+03, atom= 1480
Step=  100, Dmax= 1.7e-02 nm, Epot= -3.88698e+05 Fmax= 4.78003e+04, atom= 1480
Step=  101, Dmax= 2.1e-02 nm, Epot= -3.92915e+05 Fmax= 2.03266e+04, atom= 1480
Step=  103, Dmax= 1.2e-02 nm, Epot= -3.94530e+05 Fmax= 2.06379e+04, atom= 1480
Step=  104, Dmax= 1.5e-02 nm, Epot= -3.95805e+05 Fma

Step=  221, Dmax= 8.5e-03 nm, Epot= -5.28402e+05 Fmax= 8.63635e+03, atom= 1480
Step=  222, Dmax= 1.0e-02 nm, Epot= -5.28912e+05 Fmax= 2.52869e+04, atom= 1480
Step=  223, Dmax= 1.2e-02 nm, Epot= -5.29988e+05 Fmax= 1.51846e+04, atom= 1480
Step=  225, Dmax= 7.4e-03 nm, Epot= -5.30712e+05 Fmax= 9.12346e+03, atom= 1480
Step=  226, Dmax= 8.8e-03 nm, Epot= -5.31312e+05 Fmax= 1.98986e+04, atom= 1480
Step=  227, Dmax= 1.1e-02 nm, Epot= -5.32113e+05 Fmax= 1.51429e+04, atom= 1480
Step=  228, Dmax= 1.3e-02 nm, Epot= -5.32334e+05 Fmax= 2.66470e+04, atom= 1480
Step=  229, Dmax= 1.5e-02 nm, Epot= -5.33147e+05 Fmax= 2.38231e+04, atom= 1480
Step=  231, Dmax= 9.2e-03 nm, Epot= -5.34263e+05 Fmax= 6.47234e+03, atom= 1480
Step=  232, Dmax= 1.1e-02 nm, Epot= -5.34902e+05 Fmax= 3.00124e+04, atom= 1480
Step=  233, Dmax= 1.3e-02 nm, Epot= -5.36305e+05 Fmax= 1.35276e+04, atom= 1480
Step=  235, Dmax= 7.9e-03 nm, Epot= -5.36946e+05 Fmax= 1.26344e+04, atom= 1480
Step=  236, Dmax= 9.5e-03 nm, Epot= -5.37468e+05 Fma

Step=  353, Dmax= 1.3e-02 nm, Epot= -6.03587e+05 Fmax= 1.68642e+04, atom= 1480
Step=  355, Dmax= 7.8e-03 nm, Epot= -6.04148e+05 Fmax= 9.06085e+03, atom= 1480
Step=  356, Dmax= 9.4e-03 nm, Epot= -6.04423e+05 Fmax= 2.21237e+04, atom= 1480
Step=  357, Dmax= 1.1e-02 nm, Epot= -6.05075e+05 Fmax= 1.51550e+04, atom= 1480
Step=  359, Dmax= 6.8e-03 nm, Epot= -6.05592e+05 Fmax= 7.20658e+03, atom= 1480
Step=  360, Dmax= 8.1e-03 nm, Epot= -6.06006e+05 Fmax= 1.95543e+04, atom= 1480
Step=  361, Dmax= 9.8e-03 nm, Epot= -6.06609e+05 Fmax= 1.26862e+04, atom= 1480
Step=  362, Dmax= 1.2e-02 nm, Epot= -6.06671e+05 Fmax= 2.58239e+04, atom= 1480
Step=  363, Dmax= 1.4e-02 nm, Epot= -6.07331e+05 Fmax= 2.06146e+04, atom= 1480
Step=  365, Dmax= 8.4e-03 nm, Epot= -6.08035e+05 Fmax= 7.28249e+03, atom= 1480
Step=  366, Dmax= 1.0e-02 nm, Epot= -6.08287e+05 Fmax= 2.62482e+04, atom= 1480
Step=  367, Dmax= 1.2e-02 nm, Epot= -6.09156e+05 Fmax= 1.38543e+04, atom= 1480
Step=  369, Dmax= 7.3e-03 nm, Epot= -6.09615e+05 Fma

Step=  485, Dmax= 8.4e-03 nm, Epot= -6.57029e+05 Fmax= 1.57568e+04, atom= 1480
Step=  486, Dmax= 1.0e-02 nm, Epot= -6.57288e+05 Fmax= 1.73243e+04, atom= 1480
Step=  487, Dmax= 1.2e-02 nm, Epot= -6.57433e+05 Fmax= 2.25209e+04, atom= 1480
Step=  488, Dmax= 1.4e-02 nm, Epot= -6.57621e+05 Fmax= 2.51030e+04, atom= 1480
Step=  490, Dmax= 8.7e-03 nm, Epot= -6.58386e+05 Fmax= 3.41779e+03, atom= 1480
Step=  491, Dmax= 1.0e-02 nm, Epot= -6.58926e+05 Fmax= 3.09097e+04, atom= 1480
Step=  492, Dmax= 1.2e-02 nm, Epot= -6.59962e+05 Fmax= 1.02915e+04, atom= 1480
Step=  494, Dmax= 7.5e-03 nm, Epot= -6.60211e+05 Fmax= 1.43720e+04, atom= 1480
Step=  495, Dmax= 9.0e-03 nm, Epot= -6.60481e+05 Fmax= 1.53558e+04, atom= 1480
Step=  496, Dmax= 1.1e-02 nm, Epot= -6.60645e+05 Fmax= 2.01858e+04, atom= 1480
Step=  497, Dmax= 1.3e-02 nm, Epot= -6.60854e+05 Fmax= 2.26254e+04, atom= 1480
Step=  498, Dmax= 1.6e-02 nm, Epot= -6.60876e+05 Fmax= 2.85388e+04, atom= 1480
Step=  499, Dmax= 1.9e-02 nm, Epot= -6.60924e+05 Fma

...

Step= 4356, Dmax= 4.0e-03 nm, Epot= -1.00035e+06 Fmax= 1.95970e+03, atom= 1087
Step= 4358, Dmax= 2.4e-03 nm, Epot= -1.00037e+06 Fmax= 5.99190e+03, atom= 1087
Step= 4359, Dmax= 2.9e-03 nm, Epot= -1.00039e+06 Fmax= 3.52147e+03, atom= 1087
Step= 4360, Dmax= 3.5e-03 nm, Epot= -1.00040e+06 Fmax= 7.89301e+03, atom= 1087
Step= 4361, Dmax= 4.2e-03 nm, Epot= -1.00043e+06 Fmax= 5.80458e+03, atom= 1087
Step= 4363, Dmax= 2.5e-03 nm, Epot= -1.00045e+06 Fmax= 2.42541e+03, atom= 1087
Step= 4364, Dmax= 3.0e-03 nm, Epot= -1.00046e+06 Fmax= 7.43805e+03, atom= 1087
Step= 4365, Dmax= 3.6e-03 nm, Epot= -1.00049e+06 Fmax= 4.40743e+03, atom= 1087
Step= 4367, Dmax= 2.2e-03 nm, Epot= -1.00050e+06 Fmax= 2.68988e+03, atom= 1087
Step= 4368, Dmax= 2.6e-03 nm, Epot= -1.00052e+06 Fmax= 5.83783e+03, atom= 1087
Step= 4369, Dmax= 3.1e-03 nm, Epot= -1.00054e+06 Fmax= 4.38848e+03, atom= 1087
Step= 4370, Dmax= 3.7e-03 nm, Epot= -1.00054e+06 Fmax= 7.88168e+03, atom= 1087
Step= 4371, Dmax= 4.5e-03 nm, Epot= -1.00056e+06 Fma

Step= 4498, Dmax= 2.8e-03 nm, Epot= -1.00276e+06 Fmax= 3.22008e+03, atom= 1087
Step= 4499, Dmax= 3.3e-03 nm, Epot= -1.00276e+06 Fmax= 7.68726e+03, atom= 1087
Step= 4500, Dmax= 4.0e-03 nm, Epot= -1.00279e+06 Fmax= 5.39590e+03, atom= 1087
Step= 4502, Dmax= 2.4e-03 nm, Epot= -1.00281e+06 Fmax= 2.46479e+03, atom= 1087
Step= 4503, Dmax= 2.9e-03 nm, Epot= -1.00282e+06 Fmax= 6.95680e+03, atom= 1087
Step= 4504, Dmax= 3.4e-03 nm, Epot= -1.00284e+06 Fmax= 4.35566e+03, atom= 1087
Step= 4505, Dmax= 4.1e-03 nm, Epot= -1.00284e+06 Fmax= 9.22068e+03, atom= 1087
Step= 4506, Dmax= 4.9e-03 nm, Epot= -1.00287e+06 Fmax= 7.06507e+03, atom= 1087
Step= 4508, Dmax= 3.0e-03 nm, Epot= -1.00289e+06 Fmax= 2.69461e+03, atom= 1087
Step= 4509, Dmax= 3.6e-03 nm, Epot= -1.00290e+06 Fmax= 9.03120e+03, atom= 1087
Step= 4510, Dmax= 4.3e-03 nm, Epot= -1.00293e+06 Fmax= 5.03070e+03, atom= 1087
Step= 4512, Dmax= 2.6e-03 nm, Epot= -1.00295e+06 Fmax= 3.41827e+03, atom= 1087
Step= 4513, Dmax= 3.1e-03 nm, Epot= -1.00296e+06 Fma

Step= 4638, Dmax= 3.2e-03 nm, Epot= -1.00495e+06 Fmax= 6.40487e+03, atom= 1087
Step= 4639, Dmax= 3.8e-03 nm, Epot= -1.00497e+06 Fmax= 6.09718e+03, atom= 1087
Step= 4640, Dmax= 4.5e-03 nm, Epot= -1.00497e+06 Fmax= 8.89504e+03, atom= 1087
Step= 4641, Dmax= 5.5e-03 nm, Epot= -1.00498e+06 Fmax= 9.10886e+03, atom= 1087
Step= 4643, Dmax= 3.3e-03 nm, Epot= -1.00502e+06 Fmax= 1.70806e+03, atom= 1087
Step= 4644, Dmax= 3.9e-03 nm, Epot= -1.00503e+06 Fmax= 1.12389e+04, atom= 1087
Step= 4645, Dmax= 4.7e-03 nm, Epot= -1.00508e+06 Fmax= 4.32322e+03, atom= 1087
Step= 4647, Dmax= 2.8e-03 nm, Epot= -1.00509e+06 Fmax= 5.00775e+03, atom= 1087
Step= 4648, Dmax= 3.4e-03 nm, Epot= -1.00510e+06 Fmax= 6.19011e+03, atom= 1087
Step= 4649, Dmax= 4.1e-03 nm, Epot= -1.00511e+06 Fmax= 7.24962e+03, atom= 1087
Step= 4650, Dmax= 4.9e-03 nm, Epot= -1.00512e+06 Fmax= 8.86957e+03, atom= 1087
Step= 4651, Dmax= 5.9e-03 nm, Epot= -1.00513e+06 Fmax= 1.04880e+04, atom= 1087
Step= 4653, Dmax= 3.5e-03 nm, Epot= -1.00517e+06 Fma

Step= 4768, Dmax= 3.4e-03 nm, Epot= -1.00708e+06 Fmax= 1.72316e+03, atom= 1087
Step= 4769, Dmax= 4.0e-03 nm, Epot= -1.00709e+06 Fmax= 1.16166e+04, atom= 1087
Step= 4770, Dmax= 4.8e-03 nm, Epot= -1.00714e+06 Fmax= 4.36750e+03, atom= 1087
Step= 4772, Dmax= 2.9e-03 nm, Epot= -1.00715e+06 Fmax= 5.23089e+03, atom= 1087
Step= 4773, Dmax= 3.5e-03 nm, Epot= -1.00716e+06 Fmax= 6.28507e+03, atom= 1087
Step= 4774, Dmax= 4.2e-03 nm, Epot= -1.00717e+06 Fmax= 7.53165e+03, atom= 1087
Step= 4775, Dmax= 5.0e-03 nm, Epot= -1.00717e+06 Fmax= 9.05691e+03, atom= 1087
Step= 4776, Dmax= 6.0e-03 nm, Epot= -1.00718e+06 Fmax= 1.08334e+04, atom= 1087
Step= 4778, Dmax= 3.6e-03 nm, Epot= -1.00722e+06 Fmax= 1.08309e+03, atom= 1087
Step= 4779, Dmax= 4.3e-03 nm, Epot= -1.00725e+06 Fmax= 1.32913e+04, atom= 1087
Step= 4780, Dmax= 5.2e-03 nm, Epot= -1.00732e+06 Fmax= 3.89551e+03, atom= 1087
Step= 4782, Dmax= 3.1e-03 nm, Epot= -1.00733e+06 Fmax= 6.40647e+03, atom= 1087
Step= 4783, Dmax= 3.8e-03 nm, Epot= -1.00734e+06 Fma

Step= 4910, Dmax= 5.6e-03 nm, Epot= -1.00929e+06 Fmax= 1.01533e+04, atom= 1087
Step= 4911, Dmax= 6.7e-03 nm, Epot= -1.00929e+06 Fmax= 1.18547e+04, atom= 1087
Step= 4913, Dmax= 4.0e-03 nm, Epot= -1.00934e+06 Fmax= 1.37112e+03, atom= 1087
Step= 4914, Dmax= 4.8e-03 nm, Epot= -1.00935e+06 Fmax= 1.44279e+04, atom= 1087
Step= 4915, Dmax= 5.8e-03 nm, Epot= -1.00942e+06 Fmax= 4.58796e+03, atom= 1087
Step= 4917, Dmax= 3.5e-03 nm, Epot= -1.00943e+06 Fmax= 6.82668e+03, atom= 1087
Step= 4918, Dmax= 4.1e-03 nm, Epot= -1.00944e+06 Fmax= 6.85496e+03, atom= 1087
Step= 4920, Dmax= 2.5e-03 nm, Epot= -1.00947e+06 Fmax= 1.34631e+03, atom= 1087
Step= 4921, Dmax= 3.0e-03 nm, Epot= -1.00949e+06 Fmax= 8.51414e+03, atom= 1087
Step= 4922, Dmax= 3.6e-03 nm, Epot= -1.00952e+06 Fmax= 3.30119e+03, atom= 1087
Step= 4924, Dmax= 2.1e-03 nm, Epot= -1.00953e+06 Fmax= 3.79570e+03, atom= 1087
Step= 4925, Dmax= 2.6e-03 nm, Epot= -1.00954e+06 Fmax= 4.71475e+03, atom= 1087
Step= 4926, Dmax= 3.1e-03 nm, Epot= -1.00955e+06 Fma

In [39]:
!gmx_mpi grompp -f step6.1_minimization.mdp -o step6.1_minimization.tpr -c step6.0_minimization.gro -r step6.0_minimization.gro -p topol.top

                      :-) GROMACS - gmx grompp, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

In [40]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.1_minimization -ntomp 5 -v

                      :-) GROMACS - gmx mdrun, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
      

Step=   99, Dmax= 2.5e-03 nm, Epot= -1.01211e+06 Fmax= 2.52215e+03, atom= 1087
Step=  101, Dmax= 1.5e-03 nm, Epot= -1.01212e+06 Fmax= 2.39244e+03, atom= 1087
Step=  102, Dmax= 1.8e-03 nm, Epot= -1.01213e+06 Fmax= 3.51341e+03, atom= 1087
Step=  103, Dmax= 2.1e-03 nm, Epot= -1.01214e+06 Fmax= 3.56610e+03, atom= 1087
Step=  104, Dmax= 2.6e-03 nm, Epot= -1.01215e+06 Fmax= 4.93468e+03, atom= 1087
Step=  105, Dmax= 3.1e-03 nm, Epot= -1.01216e+06 Fmax= 5.26397e+03, atom= 1087
Step=  106, Dmax= 3.7e-03 nm, Epot= -1.01217e+06 Fmax= 6.97246e+03, atom= 1087
Step=  107, Dmax= 4.4e-03 nm, Epot= -1.01217e+06 Fmax= 7.71605e+03, atom= 1087
Step=  109, Dmax= 2.7e-03 nm, Epot= -1.01221e+06 Fmax= 1.10920e+03, atom= 1087
Step=  110, Dmax= 3.2e-03 nm, Epot= -1.01222e+06 Fmax= 9.44383e+03, atom= 1087
Step=  111, Dmax= 3.8e-03 nm, Epot= -1.01227e+06 Fmax= 3.25106e+03, atom= 1087
Step=  113, Dmax= 2.3e-03 nm, Epot= -1.01228e+06 Fmax= 4.36358e+03, atom= 1087
Step=  114, Dmax= 2.8e-03 nm, Epot= -1.01229e+06 Fma

Step=  238, Dmax= 2.4e-03 nm, Epot= -1.01405e+06 Fmax= 4.04121e+03, atom= 1087
Step=  239, Dmax= 2.8e-03 nm, Epot= -1.01406e+06 Fmax= 5.35490e+03, atom= 1087
Step=  240, Dmax= 3.4e-03 nm, Epot= -1.01407e+06 Fmax= 5.91975e+03, atom= 1087
Step=  241, Dmax= 4.1e-03 nm, Epot= -1.01407e+06 Fmax= 7.61244e+03, atom= 1087
Step=  242, Dmax= 4.9e-03 nm, Epot= -1.01407e+06 Fmax= 8.61907e+03, atom= 1087
Step=  244, Dmax= 3.0e-03 nm, Epot= -1.01412e+06 Fmax= 1.10701e+03, atom= 1087
Step=  245, Dmax= 3.5e-03 nm, Epot= -1.01412e+06 Fmax= 1.06097e+04, atom= 1087
Step=  246, Dmax= 4.3e-03 nm, Epot= -1.01418e+06 Fmax= 3.41159e+03, atom= 1087
Step=  248, Dmax= 2.6e-03 nm, Epot= -1.01419e+06 Fmax= 5.00139e+03, atom= 1087
Step=  249, Dmax= 3.1e-03 nm, Epot= -1.01420e+06 Fmax= 5.09840e+03, atom= 1087
Step=  250, Dmax= 3.7e-03 nm, Epot= -1.01420e+06 Fmax= 7.01876e+03, atom= 1087
Step=  251, Dmax= 4.4e-03 nm, Epot= -1.01421e+06 Fmax= 7.52579e+03, atom= 1087
Step=  253, Dmax= 2.6e-03 nm, Epot= -1.01424e+06 Fma

Step=  380, Dmax= 3.9e-03 nm, Epot= -1.01602e+06 Fmax= 2.90464e+03, atom= 1087
Step=  382, Dmax= 2.3e-03 nm, Epot= -1.01603e+06 Fmax= 4.84313e+03, atom= 1087
Step=  383, Dmax= 2.8e-03 nm, Epot= -1.01604e+06 Fmax= 4.46127e+03, atom= 1087
Step=  384, Dmax= 3.4e-03 nm, Epot= -1.01604e+06 Fmax= 6.70090e+03, atom= 1087
Step=  385, Dmax= 4.1e-03 nm, Epot= -1.01605e+06 Fmax= 6.69872e+03, atom= 1087
Step=  387, Dmax= 2.4e-03 nm, Epot= -1.01608e+06 Fmax= 1.35037e+03, atom= 1087
Step=  388, Dmax= 2.9e-03 nm, Epot= -1.01609e+06 Fmax= 8.28664e+03, atom= 1087
Step=  389, Dmax= 3.5e-03 nm, Epot= -1.01612e+06 Fmax= 3.29476e+03, atom= 1087
Step=  391, Dmax= 2.1e-03 nm, Epot= -1.01613e+06 Fmax= 3.64803e+03, atom= 1087
Step=  392, Dmax= 2.5e-03 nm, Epot= -1.01614e+06 Fmax= 4.68748e+03, atom= 1087
Step=  393, Dmax= 3.0e-03 nm, Epot= -1.01615e+06 Fmax= 5.31407e+03, atom= 1087
Step=  394, Dmax= 3.6e-03 nm, Epot= -1.01615e+06 Fmax= 6.68705e+03, atom= 1087
Step=  395, Dmax= 4.4e-03 nm, Epot= -1.01616e+06 Fma

Step=  523, Dmax= 3.2e-03 nm, Epot= -1.01776e+06 Fmax= 2.67496e+03, atom= 1087
Step=  525, Dmax= 1.9e-03 nm, Epot= -1.01777e+06 Fmax= 3.72455e+03, atom= 1087
Step=  526, Dmax= 2.3e-03 nm, Epot= -1.01778e+06 Fmax= 3.95526e+03, atom= 1087
Step=  527, Dmax= 2.8e-03 nm, Epot= -1.01778e+06 Fmax= 5.25898e+03, atom= 1087
Step=  528, Dmax= 3.4e-03 nm, Epot= -1.01779e+06 Fmax= 5.79563e+03, atom= 1087
Step=  529, Dmax= 4.0e-03 nm, Epot= -1.01779e+06 Fmax= 7.47694e+03, atom= 1087
Step=  530, Dmax= 4.8e-03 nm, Epot= -1.01780e+06 Fmax= 8.44104e+03, atom= 1087
Step=  532, Dmax= 2.9e-03 nm, Epot= -1.01784e+06 Fmax= 1.09861e+03, atom= 1087
Step=  533, Dmax= 3.5e-03 nm, Epot= -1.01784e+06 Fmax= 1.03851e+04, atom= 1087
Step=  534, Dmax= 4.2e-03 nm, Epot= -1.01790e+06 Fmax= 3.36254e+03, atom= 1087
Step=  536, Dmax= 2.5e-03 nm, Epot= -1.01790e+06 Fmax= 4.89037e+03, atom= 1087
Step=  537, Dmax= 3.0e-03 nm, Epot= -1.01791e+06 Fmax= 5.01510e+03, atom= 1087
Step=  538, Dmax= 3.6e-03 nm, Epot= -1.01792e+06 Fma

...

Step= 4457, Dmax= 1.7e-03 nm, Epot= -1.05123e+06 Fmax= 3.37443e+03, atom= 1480
Step= 4458, Dmax= 2.1e-03 nm, Epot= -1.05123e+06 Fmax= 3.43735e+03, atom= 1480
Step= 4459, Dmax= 2.5e-03 nm, Epot= -1.05124e+06 Fmax= 4.72448e+03, atom= 1480
Step= 4460, Dmax= 3.0e-03 nm, Epot= -1.05124e+06 Fmax= 5.08400e+03, atom= 1480
Step= 4461, Dmax= 3.6e-03 nm, Epot= -1.05124e+06 Fmax= 6.66891e+03, atom= 1480
Step= 4462, Dmax= 4.3e-03 nm, Epot= -1.05124e+06 Fmax= 7.45559e+03, atom= 1480
Step= 4464, Dmax= 2.6e-03 nm, Epot= -1.05127e+06 Fmax= 1.02153e+03, atom= 1480
Step= 4466, Dmax= 1.5e-03 nm, Epot= -1.05127e+06 Fmax= 4.05908e+03, atom= 1480
Step= 4467, Dmax= 1.8e-03 nm, Epot= -1.05128e+06 Fmax= 2.03542e+03, atom= 1480
Step= 4468, Dmax= 2.2e-03 nm, Epot= -1.05128e+06 Fmax= 5.29329e+03, atom= 1480
Step= 4469, Dmax= 2.7e-03 nm, Epot= -1.05129e+06 Fmax= 3.48551e+03, atom= 1480
Step= 4471, Dmax= 1.6e-03 nm, Epot= -1.05130e+06 Fmax= 1.78383e+03, atom= 1480
Step= 4472, Dmax= 1.9e-03 nm, Epot= -1.05130e+06 Fma

Step= 4591, Dmax= 1.6e-03 nm, Epot= -1.05201e+06 Fmax= 4.22696e+03, atom= 1480
Step= 4592, Dmax= 1.9e-03 nm, Epot= -1.05202e+06 Fmax= 2.04718e+03, atom= 1480
Step= 4593, Dmax= 2.3e-03 nm, Epot= -1.05202e+06 Fmax= 5.46672e+03, atom= 1480
Step= 4594, Dmax= 2.7e-03 nm, Epot= -1.05203e+06 Fmax= 3.56578e+03, atom= 1480
Step= 4596, Dmax= 1.6e-03 nm, Epot= -1.05204e+06 Fmax= 1.85218e+03, atom= 1480
Step= 4597, Dmax= 2.0e-03 nm, Epot= -1.05204e+06 Fmax= 4.65905e+03, atom= 1480
Step= 4598, Dmax= 2.4e-03 nm, Epot= -1.05205e+06 Fmax= 3.14161e+03, atom= 1480
Step= 4600, Dmax= 1.4e-03 nm, Epot= -1.05206e+06 Fmax= 1.53961e+03, atom= 1480
Step= 4601, Dmax= 1.7e-03 nm, Epot= -1.05206e+06 Fmax= 4.07069e+03, atom= 1480
Step= 4602, Dmax= 2.0e-03 nm, Epot= -1.05207e+06 Fmax= 2.67348e+03, atom= 1480
Step= 4604, Dmax= 1.2e-03 nm, Epot= -1.05207e+06 Fmax= 1.36953e+03, atom= 1480
Step= 4605, Dmax= 1.5e-03 nm, Epot= -1.05208e+06 Fmax= 3.49168e+03, atom= 1480
Step= 4606, Dmax= 1.8e-03 nm, Epot= -1.05209e+06 Fma

Step= 4722, Dmax= 2.0e-03 nm, Epot= -1.05272e+06 Fmax= 5.94277e+03, atom= 1480
Step= 4723, Dmax= 2.4e-03 nm, Epot= -1.05274e+06 Fmax= 2.08121e+03, atom= 1480
Step= 4725, Dmax= 1.5e-03 nm, Epot= -1.05274e+06 Fmax= 2.72814e+03, atom= 1480
Step= 4726, Dmax= 1.8e-03 nm, Epot= -1.05275e+06 Fmax= 3.05309e+03, atom= 1480
Step= 4727, Dmax= 2.1e-03 nm, Epot= -1.05275e+06 Fmax= 3.87415e+03, atom= 1480
Step= 4728, Dmax= 2.5e-03 nm, Epot= -1.05276e+06 Fmax= 4.45038e+03, atom= 1480
Step= 4729, Dmax= 3.0e-03 nm, Epot= -1.05276e+06 Fmax= 5.52483e+03, atom= 1480
Step= 4730, Dmax= 3.6e-03 nm, Epot= -1.05276e+06 Fmax= 6.46242e+03, atom= 1480
Step= 4732, Dmax= 2.2e-03 nm, Epot= -1.05278e+06 Fmax= 7.29301e+02, atom= 1480
Step= 4733, Dmax= 2.6e-03 nm, Epot= -1.05279e+06 Fmax= 7.91150e+03, atom= 1480
Step= 4734, Dmax= 3.1e-03 nm, Epot= -1.05281e+06 Fmax= 2.43865e+03, atom= 1480
Step= 4736, Dmax= 1.9e-03 nm, Epot= -1.05281e+06 Fmax= 3.78089e+03, atom= 1480
Step= 4737, Dmax= 2.3e-03 nm, Epot= -1.05282e+06 Fma

Step= 4854, Dmax= 3.1e-03 nm, Epot= -1.05347e+06 Fmax= 5.49338e+03, atom= 1480
Step= 4856, Dmax= 1.9e-03 nm, Epot= -1.05349e+06 Fmax= 6.69630e+02, atom= 1480
Step= 4857, Dmax= 2.2e-03 nm, Epot= -1.05350e+06 Fmax= 6.73879e+03, atom= 1480
Step= 4858, Dmax= 2.7e-03 nm, Epot= -1.05351e+06 Fmax= 2.12902e+03, atom= 1480
Step= 4860, Dmax= 1.6e-03 nm, Epot= -1.05352e+06 Fmax= 3.20075e+03, atom= 1480
Step= 4861, Dmax= 1.9e-03 nm, Epot= -1.05352e+06 Fmax= 3.18183e+03, atom= 1480
Step= 4862, Dmax= 2.3e-03 nm, Epot= -1.05353e+06 Fmax= 4.48957e+03, atom= 1480
Step= 4863, Dmax= 2.8e-03 nm, Epot= -1.05353e+06 Fmax= 4.70033e+03, atom= 1480
Step= 4865, Dmax= 1.7e-03 nm, Epot= -1.05354e+06 Fmax= 8.16580e+02, atom= 1480
Step= 4866, Dmax= 2.0e-03 nm, Epot= -1.05355e+06 Fmax= 5.78766e+03, atom= 1480
Step= 4867, Dmax= 2.4e-03 nm, Epot= -1.05356e+06 Fmax= 2.15933e+03, atom= 1480
Step= 4869, Dmax= 1.4e-03 nm, Epot= -1.05357e+06 Fmax= 2.60286e+03, atom= 1480
Step= 4870, Dmax= 1.7e-03 nm, Epot= -1.05357e+06 Fma

Step= 5000, Dmax= 1.8e-03 nm, Epot= -1.05431e+06 Fmax= 8.36278e+02, atom= 1480

Energy minimization reached the maximum number of steps before the forces
reached the requested precision Fmax < 10.

writing lowest energy coordinates.

Back Off! I just backed up step6.1_minimization.gro to ./#step6.1_minimization.gro.1#

Steepest Descents did not converge to Fmax < 10 in 5001 steps.
Potential Energy  = -1.0543095e+06
Maximum force     =  8.3627826e+02 on atom 1480
Norm of force     =  1.5305838e+01

GROMACS reminds you: "This isn't right. This isn't even wrong." (Wolfgang Pauli)



# Equilibration Steps - NVT

In [41]:
!gmx_mpi grompp -f step6.2_equilibration.mdp -o step6.2_equilibration.tpr -c step6.1_minimization.gro -r step6.1_minimization.gro -p topol.top

                      :-) GROMACS - gmx grompp, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

In [42]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.2_equilibration -ntomp 5 -gpu_id 0 -v

                      :-) GROMACS - gmx mdrun, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
      

# Equilibration Steps - NPT

In [43]:
!gmx_mpi grompp -f step6.3_equilibration.mdp -o step6.3_equilibration.tpr -c step6.2_equilibration.gro -r step6.2_equilibration.gro -p topol.top

                      :-) GROMACS - gmx grompp, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

In [44]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.3_equilibration -ntomp 5 -gpu_id 0 -v

                      :-) GROMACS - gmx mdrun, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
      

In [45]:
!gmx_mpi grompp -f step6.4_equilibration.mdp -o step6.4_equilibration.tpr -c step6.3_equilibration.gro -r step6.3_equilibration.gro -p topol.top

                      :-) GROMACS - gmx grompp, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

In [46]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.4_equilibration -ntomp 5 -gpu_id 0 -v

                      :-) GROMACS - gmx mdrun, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
      

In [47]:
!gmx_mpi grompp -f step6.5_equilibration.mdp -o step6.5_equilibration.tpr -c step6.4_equilibration.gro -r step6.4_equilibration.gro -p topol.top

                      :-) GROMACS - gmx grompp, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

In [48]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.5_equilibration -ntomp 5 -gpu_id 0 -v

                      :-) GROMACS - gmx mdrun, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
      

In [49]:
!gmx_mpi grompp -f step6.6_equilibration.mdp -o step6.6_equilibration.tpr -c step6.5_equilibration.gro -r step6.5_equilibration.gro -p topol.top

                      :-) GROMACS - gmx grompp, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

In [50]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.6_equilibration -ntomp 5 -gpu_id 0 -v

                      :-) GROMACS - gmx mdrun, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
      

# MD run 

In [51]:
!gmx_mpi grompp -f step7_production.mdp -o step7_production.tpr -c step6.6_equilibration.gro -r step6.6_equilibration.gro -p topol.top

                      :-) GROMACS - gmx grompp, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

In [ ]:
!gmx_mpi mdrun -rdd 3.0 -deffnm step7_production -ntomp 12 -gpu_id 0 -v

                      :-) GROMACS - gmx mdrun, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
      

step 66440200, will finish Fri Dec 16 10:52:02 2022 will finish Fri Dec 16 03:49:48 2022, will finish Fri Dec 16 04:42:48 2022Fri Dec 16 04:44:46 2022, will finish Fri Dec 16 04:44:48 2022, will finish Fri Dec 16 04:45:44 2022, will finish Fri Dec 16 04:48:05 2022, will finish Fri Dec 16 05:01:13 2022, will finish Fri Dec 16 05:08:50 2022Fri Dec 16 06:31:53 2022, will finish Fri Dec 16 06:33:34 2022, will finish Fri Dec 16 06:47:47 2022, will finish Fri Dec 16 07:38:29 2022Fri Dec 16 07:48:30 2022, will finish Fri Dec 16 08:37:02 2022, will finish Fri Dec 16 09:08:53 2022, will finish Fri Dec 16 09:08:49 2022, will finish Fri Dec 16 09:09:59 2022, will finish Fri Dec 16 09:13:11 2022, will finish Fri Dec 16 09:17:26 2022Fri Dec 16 09:17:34 2022Fri Dec 16 09:19:13 2022, will finish Fri Dec 16 09:38:00 2022, will finish Fri Dec 16 09:39:14 2022, will finish Fri Dec 16 09:41:12 2022, will finish Fri Dec 16 09:41:10 2022, will finish Fri Dec 16 09:42:45 2022, will finish Fri Dec 16 09:44:1

step 79535200, will finish Fri Dec 16 11:02:39 2022Fri Dec 16 10:52:02 2022, will finish Fri Dec 16 10:52:00 2022, will finish Fri Dec 16 10:52:07 2022, will finish Fri Dec 16 10:52:07 2022, will finish Fri Dec 16 10:52:07 2022, will finish Fri Dec 16 10:52:07 2022, will finish Fri Dec 16 10:52:07 202266566000, will finish Fri Dec 16 10:52:07 2022, will finish Fri Dec 16 10:52:06 2022, will finish Fri Dec 16 10:52:05 2022, will finish Fri Dec 16 10:52:05 2022, will finish Fri Dec 16 10:52:05 2022, will finish Fri Dec 16 10:52:04 2022, will finish Fri Dec 16 10:51:59 2022, will finish Fri Dec 16 10:52:01 2022Fri Dec 16 10:52:20 2022, will finish Fri Dec 16 10:52:30 2022, will finish Fri Dec 16 10:52:33 2022, will finish Fri Dec 16 10:52:44 2022, will finish Fri Dec 16 10:52:44 2022, will finish Fri Dec 16 10:52:37 2022, will finish Fri Dec 16 10:52:37 2022, will finish Fri Dec 16 10:52:33 2022, will finish Fri Dec 16 10:52:26 2022, will finish Fri Dec 16 10:52:27 2022, will finish Fri D

step 91024000, will finish Fri Dec 16 11:01:30 2022, will finish Fri Dec 16 11:02:35 2022, will finish Fri Dec 16 11:02:34 2022, will finish Fri Dec 16 11:02:37 2022, will finish Fri Dec 16 11:02:54 2022, will finish Fri Dec 16 11:03:22 2022Fri Dec 16 11:03:57 2022Fri Dec 16 11:04:02 2022, will finish Fri Dec 16 11:04:49 2022, will finish Fri Dec 16 11:04:49 2022, will finish Fri Dec 16 11:04:49 2022, will finish Fri Dec 16 11:04:51 2022, will finish Fri Dec 16 11:04:54 2022, will finish Fri Dec 16 11:04:54 2022, will finish Fri Dec 16 11:05:02 2022, will finish Fri Dec 16 11:05:02 2022Fri Dec 16 11:05:06 2022, will finish Fri Dec 16 11:05:04 2022, will finish Fri Dec 16 11:05:04 2022, will finish Fri Dec 16 11:05:07 2022, will finish Fri Dec 16 11:05:28 2022, will finish Fri Dec 16 11:05:34 2022Fri Dec 16 11:05:34 2022, will finish Fri Dec 16 11:05:22 2022, will finish Fri Dec 16 11:05:23 2022Fri Dec 16 11:05:21 2022, will finish Fri Dec 16 11:05:17 2022, will finish Fri Dec 16 11:05:

step 96557100, will finish Fri Dec 16 11:06:35 2022, will finish Fri Dec 16 11:01:28 2022, will finish Fri Dec 16 11:01:28 2022, will finish Fri Dec 16 11:01:28 2022, will finish Fri Dec 16 11:01:28 2022, will finish Fri Dec 16 11:01:30 2022, will finish Fri Dec 16 11:01:30 2022, will finish Fri Dec 16 11:01:35 2022, will finish Fri Dec 16 11:01:36 2022, will finish Fri Dec 16 11:01:36 2022, will finish Fri Dec 16 11:01:36 2022, will finish Fri Dec 16 11:01:36 2022, will finish Fri Dec 16 11:01:37 2022, will finish Fri Dec 16 11:01:40 2022Fri Dec 16 11:01:40 2022Fri Dec 16 11:01:43 2022, will finish Fri Dec 16 11:01:43 2022, will finish Fri Dec 16 11:01:46 2022, will finish Fri Dec 16 11:01:48 2022, will finish Fri Dec 16 11:01:48 2022, will finish Fri Dec 16 11:01:45 2022, will finish Fri Dec 16 11:01:45 2022, will finish Fri Dec 16 11:01:45 2022, will finish Fri Dec 16 11:01:45 2022, will finish Fri Dec 16 11:01:44 2022, will finish Fri Dec 16 11:01:44 2022, will finish Fri Dec 16 11

step 101673900, will finish Fri Dec 16 11:07:45 2022 will finish Fri Dec 16 11:06:25 2022, will finish Fri Dec 16 11:06:21 2022, will finish Fri Dec 16 11:06:18 2022, will finish Fri Dec 16 11:06:13 2022, will finish Fri Dec 16 11:06:13 2022, will finish Fri Dec 16 11:06:11 2022, will finish Fri Dec 16 11:06:08 2022, will finish Fri Dec 16 11:06:08 2022, will finish Fri Dec 16 11:06:09 2022, will finish Fri Dec 16 11:06:14 2022, will finish Fri Dec 16 11:06:18 2022, will finish Fri Dec 16 11:06:29 2022, will finish Fri Dec 16 11:06:31 2022, will finish Fri Dec 16 11:06:38 2022, will finish Fri Dec 16 11:06:35 2022, will finish Fri Dec 16 11:06:35 2022, will finish Fri Dec 16 11:06:30 2022, will finish Fri Dec 16 11:06:32 2022, will finish Fri Dec 16 11:06:30 2022, will finish Fri Dec 16 11:06:27 2022Fri Dec 16 11:06:28 2022, will finish Fri Dec 16 11:06:29 2022, will finish Fri Dec 16 11:06:30 2022Fri Dec 16 11:06:32 2022Fri Dec 16 11:06:34 2022, will finish Fri Dec 16 11:06:34 2022, w

step 126164100, will finish Fri Dec 16 07:32:37 2022, will finish Fri Dec 16 11:07:42 2022, will finish Fri Dec 16 11:07:36 2022, will finish Fri Dec 16 11:07:36 2022, will finish Fri Dec 16 11:07:35 2022, will finish Fri Dec 16 11:07:35 2022Fri Dec 16 11:07:36 2022, will finish Fri Dec 16 11:07:35 2022, will finish Fri Dec 16 11:07:40 2022, will finish Fri Dec 16 11:07:41 2022, will finish Fri Dec 16 11:07:54 2022, will finish Fri Dec 16 11:08:06 2022, will finish Fri Dec 16 11:08:03 2022, will finish Fri Dec 16 11:08:03 2022, will finish Fri Dec 16 11:08:02 2022, will finish Fri Dec 16 11:08:03 2022, will finish Fri Dec 16 11:08:06 2022, will finish Fri Dec 16 11:08:09 2022, will finish Fri Dec 16 11:08:17 2022, will finish Fri Dec 16 11:08:18 2022, will finish Fri Dec 16 11:08:27 2022, will finish Fri Dec 16 11:08:28 2022, will finish Fri Dec 16 11:08:38 2022, will finish Fri Dec 16 11:08:39 2022, will finish Fri Dec 16 11:08:41 2022, will finish Fri Dec 16 11:08:41 2022, will finis

step 491252600, will finish Thu Dec 15 18:43:18 2022, will finish Fri Dec 16 07:32:36 2022, will finish Fri Dec 16 07:29:54 2022, will finish Fri Dec 16 07:27:14 2022, will finish Fri Dec 16 07:24:53 2022, will finish Fri Dec 16 07:24:53 2022, will finish Fri Dec 16 07:24:52 2022, will finish Fri Dec 16 07:24:49 2022, will finish Fri Dec 16 07:24:48 2022, will finish Fri Dec 16 07:24:48 2022, will finish Fri Dec 16 07:24:47 2022, will finish Fri Dec 16 07:24:47 2022, will finish Fri Dec 16 07:22:07 2022, will finish Fri Dec 16 07:22:07 2022, will finish Fri Dec 16 07:20:57 2022, will finish Fri Dec 16 07:18:07 2022, will finish Fri Dec 16 07:18:07 2022, will finish Fri Dec 16 07:18:07 2022, will finish Fri Dec 16 07:15:31 2022, will finish Fri Dec 16 07:14:20 2022, will finish Fri Dec 16 07:14:20 2022, will finish Fri Dec 16 07:14:20 2022, will finish Fri Dec 16 07:14:19 2022, will finish Fri Dec 16 07:14:18 2022, will finish Fri Dec 16 07:14:18 2022, will finish Fri Dec 16 07:14:18 20

In [ ]:
!gmx_mpi check -f step7_production.xtc

In [ ]:
ls

# make an Index

In [ ]:
#get the number of the first and last atoms of the receptor and peptide
!cat system.gro
#!cat system.gro

In [ ]:
#cg
!echo -e "a 1-725  \nname 22 rec \na 726-750 \nname 23 pep \nq" |gmx make_ndx -f system.gro -o index.ndx

# Center the the box
    - We remove PBC
    - output = step7_production.pbc.xtc

In [ ]:
ls *trr

In [ ]:
!echo 17 0|gmx_mpi trjconv -f step7_production.xtc -o step7_production.pbc.xtc -s step7_production.tpr -pbc mol -center -n index.ndx 

# Trajectory Alignment
    - We align the traj to the CG system to remove translation and rotation
    - output = traj_aligned_to_CG.trr

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import contacts, align

finp_ref  = 'system.gro'                     # reference
finp_trj  = 'step7_production.pbc.xtc'       # traj


receptor = 'name BB and resid 40-337'


U   = mda.Universe(finp_ref, finp_trj)       #Open Traj
ref = mda.Universe(finp_ref)                 #Open ref file 

# Align traj to reference using atmgroup receptor
align.AlignTraj( U, ref, select=receptor, in_memory=True).run()
'Done Boss'

# save aligned traj
system_aligned = U.select_atoms("all")
with mda.Writer("traj_aligned_to_CG.trr", system_aligned.n_atoms) as W:
    for ts in U.trajectory:
        W.write(system_aligned)

# NOW READY TO BE ANALYZED!!